In [1]:
import nest_asyncio
nest_asyncio.apply()

In [53]:
import asyncio
import re
import time
import os

from IPython.display import display, clear_output
from novelai_api.NovelAI_API import NovelAIAPI
from novelai_api.Preset import Model, Preset
from novelai_api.GlobalSettings import GlobalSettings
from novelai_api.Tokenizer import Tokenizer
from novelai_api.utils import b64_to_tokens
from novelai_api.BiasGroup import BiasGroup

import pandas as pd

In [56]:
# Settings

# Set method for authentication
# Posible options are:
# "enter_key" (enter your access key)
# "enter_login" (enter your username & pw)
# "enter_token" (enter your access token)
# "env_key" (read access key from environment variable NAI_KEY)
# "env_login" (read user from environment variable NAI_USERNAME and password from NAI_PASSWORD)
# "env_token"
auth_method = "env_token"
delay_time = 2
generation_timeout = 60
max_failed_gens = 3
candidates_goal = 10
bias_strength_inc = -0.1
bias_phrase_dict = {}

In [18]:
auth = False
env = os.environ

# Init variable for login method
if auth_method == "enter_key":
    auth = input("Enter your NovelAI access key: ")
if auth_method == "enter_token":
    auth = input("Enter your NovelAI access token: ")
elif auth_method == "enter_login":
    auth = {}
    auth["user"] = input("Enter your NovelAI username: ")
    auth["pw"] = input("Enter your NovelAI password: ")
elif auth_method == "env_key":
    auth = env["NAI_KEY"]
elif auth_method == "env_token":
    auth = env["NAI_TOKEN"]
elif auth_method == "env_login":
    auth = {}
    auth["user"] = env["NAI_USERNAME"]
    auth["pw"] = env["NAI_PASSWORD"]
else:
    raise RuntimeError("Invalid value for 'auth_method'. Must be one of 'enter_key', 'enter_token', 'enter_login', env_key', 'env_token' or 'env_login")


In [19]:
async def nai_login (api, auth_method, auth):

    if auth_method == "enter_key" or auth_method == "env_key":
        await api.high_level.login_from_key(auth)
    elif auth_method == "enter_token" or auth_method == "env_token":
        await api.high_level.login_with_token(auth)
    elif auth_method == "enter_login" or auth_method == "env_login":
        await api.high_level.login(auth["user"], auth["pw"])

In [71]:
async def gen_attg_candidate(model=Model.Clio, prompt="[ Genre:", stop_sequences=[",", ";", " ]"], cut_stop_seq = True, auth_method="env_token", auth = None, bias_groups = None):
    # Initialize the NovelAI API
    api = NovelAIAPI()

    try:
        # Ensure you're logged in
        await nai_login(api, auth_method, auth)

        # Use the official "Edgewise" preset
        preset = Preset.from_official(model, "Edgewise")
        
        # Tokenize the stop sequences and set them for the preset
        stop_sequences_tokenized = [Tokenizer.encode(model, seq) for seq in stop_sequences]
        preset["stop_sequences"] = stop_sequences_tokenized
        
        # Create default global settings
        global_settings = GlobalSettings()

        gen = await api.high_level.generate(prompt, model, preset, global_settings, None, bias_groups, None)
        
        # After generating the text, remove the stop sequence
        generated_text = Tokenizer.decode(model, b64_to_tokens(gen["output"]))
        if cut_stop_seq:
            for seq in stop_sequences:
                generated_text = re.sub(re.escape(seq) + "$", "", generated_text).strip()
    
        return generated_text
    
    except Exception as e:
        raise Exception(f"Error generating text: {e}")
    
def update_bias_groups(phrase, bias_phrase_dict, bias_strength_inc, bias_groups):
    # Update the bias strength for the phrase or add it if it's not in the dict
    if phrase in bias_phrase_dict:
        bias_phrase_dict[phrase] += bias_strength_inc
    else:
        bias_phrase_dict[phrase] = bias_strength_inc

    # Clear the existing bias groups
    bias_groups.clear()

    # Regenerate the bias groups based on the updated bias_phrase_dict
    for phrase, strength in bias_phrase_dict.items():
        bg = BiasGroup(strength)
        bg.add(phrase)
        bias_groups.append(bg)


In [76]:
# Initialize an empty DataFrame
df = pd.DataFrame(columns=["phrase", "count", "last_bias"])

# Counter for total generations
total_generations = 0

# set phrase biases
bias_groups = []
for phrase, strength in bias_phrase_dict.items():
    bg = BiasGroup(strength)
    bg.add(phrase)
    bias_groups.append(bg)

# Counter for unsuccessful generation attempts
unsuccessful_attempts = 0

# Loop until you have candidate_goal unique phrases
while len(df) < candidates_goal:
    total_generations += 1

    ## Clear the previous output
    clear_output(wait=True)
    
    print(f"Gen {total_generations}: Trying to gen phrase {len(df)+1}/{candidates_goal}...")
    ## print(f"Current bias groups {bias_phrase_dict}")
    
    try:
        phrase = await asyncio.wait_for(gen_attg_candidate(model=Model.Clio, prompt="[ Genre:", auth_method=auth_method, auth=auth, bias_groups=bias_groups), timeout=generation_timeout)

        # Check if the phrase is already in the DataFrame
        if phrase in df["phrase"].values:
            df.loc[df["phrase"] == phrase, "count"] += 1
            df.loc[df["phrase"] == phrase, "last_bias"] = bias_phrase_dict.get(phrase, bias_strength_inc)
            print(f"Phrase '{phrase}' already exists. Incrementing count and changing bias by {bias_strength_inc}.")

            # Update the bias groups since the phrase was generated again
            update_bias_groups(phrase, bias_phrase_dict, bias_strength_inc, bias_groups)
        else:
            df.loc[len(df)] = [phrase, 1, bias_phrase_dict.get(phrase, bias_strength_inc)]
            print(f"Added new phrase: '{phrase}'")

        # Reset the unsuccessful_attempts counter if generation was successful
        unsuccessful_attempts = 0

    except asyncio.TimeoutError:
        print("Generation took too long. Retrying...")
        unsuccessful_attempts += 1
        if unsuccessful_attempts >= 3:
            print("3 unsuccessful generation attempts. Aborting candidate search.")
            break
    except Exception as e:
        if "Anonymous quota reached" in str(e):
            print(f"Error: {e}")
            print("Anonymous rate limit reached. This indicates you are not properly authenticated. Check your authentication method. Aborting candidate search.")
            break
        else:
            print(f"Error: {e}")
            # import traceback
            # traceback.print_exc()
            print("Aborting candidate search")
            break

    # Wait for delay_time seconds before the next generation attempt
    time.sleep(delay_time)

print("\nCandidate search complete!")
print(df)

Gen 1: Trying to gen phrase 1/10...
Added new phrase: 'Cyberpunk'
Gen 2: Trying to gen phrase 2/10...
Added new phrase: 'LitRPG'
Gen 3: Trying to gen phrase 3/10...
Phrase 'LitRPG' already exists. Incrementing count and changing bias by -0.1.
Gen 4: Trying to gen phrase 3/10...
Added new phrase: 'Science fiction'
Gen 5: Trying to gen phrase 4/10...
Phrase 'LitRPG' already exists. Incrementing count and changing bias by -0.1.
Gen 6: Trying to gen phrase 4/10...
Added new phrase: 'science fiction'
Gen 7: Trying to gen phrase 5/10...
Added new phrase: 'Action'
Gen 8: Trying to gen phrase 6/10...
Added new phrase: 'Fantasy'
Gen 9: Trying to gen phrase 7/10...
Phrase 'Fantasy' already exists. Incrementing count and changing bias by -0.1.
Gen 10: Trying to gen phrase 7/10...
Phrase 'Fantasy' already exists. Incrementing count and changing bias by -0.1.
Gen 11: Trying to gen phrase 7/10...
Phrase 'LitRPG' already exists. Incrementing count and changing bias by -0.1.
Gen 12: Trying to gen phra

In [75]:
bias_phrase_dict

{'Action': -0.30000000000000004,
 'Fantasy': -0.2,
 'LitRPG': -3.3000000000000016,
 'science fiction': -0.30000000000000004,
 'Epic fantasy': -0.1,
 'Cyberpunk': -0.1}